<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [162]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [163]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
Using competition: 26jun23
  teamId  teamName                   submissionDate       score    
--------  -------------------------  -------------------  -------  
10612701  Никита Угрюмов             2023-06-28 23:59:12  0.87976  
10612695  Ne1tr0N [Maxim Mashtaler]  2023-06-28 22:04:00  0.87931  
10612706  Michael Kondrashov         2023-06-29 13:39:23  0.87899  
10612715  Stepan Vysotsky            2023-06-29 12:18:18  0.87687  
10616508  Ulyana Klushova            2023-06-28 20:38:57  0.87485  
10612698  Alexandr Boldirev          2023-06-29 18:33:01  0.87473  
10614521  Konstantin Zeigman         2023-06-29 13:56:25  0.87360  
10612681  Maria Alyushina            2023-06-29 18:43:03  0.87097  
10612680  esheshka                   2023-06-29 14:43:48  0.86811  
10614699  Dmitriy Geyzha             2023-06-28 12:50:43  0.86653  
10617156  Gleb Goloshchapov          2023-06-29 12:52:30  0.864

In [164]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 1.58 s, sys: 89.9 ms, total: 1.67 s
Wall time: 25.7 s


In [165]:
!pip install catboost
!pip install lightgbm
!pip install xgboost

from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier as SC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import LinearSVC as LSVC
from sklearn.linear_model import SGDClassifier as SGDC
from catboost import CatBoostClassifier as CBC
from lightgbm import LGBMClassifier as LGBMC

from sklearn.model_selection import RepeatedStratifiedKFold as RSKF

from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.model_selection import GridSearchCV as GSCV
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.calibration import CalibratedClassifierCV as CCCV


In [166]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc')

CPU times: user 6.12 ms, sys: 0 ns, total: 6.12 ms
Wall time: 10.8 ms


In [167]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc');

CPU times: user 6.14 ms, sys: 0 ns, total: 6.14 ms
Wall time: 7.14 ms


In [168]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')

CPU times: user 7.72 ms, sys: 987 µs, total: 8.7 ms
Wall time: 10.1 ms


cid,0,1,2,3,4,...,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0


In [169]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm'])

CPU times: user 9.75 s, sys: 1.04 s, total: 10.8 s
Wall time: 17.6 s


In [170]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 4)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


⏳ started. You have 120 sec. Good luck!


# **Interesting**

In [259]:
df_test = dfTrx.join(gender, on='cid', how='inner')
df_test = df_test[~df_test.gender.isna()]

In [260]:
#@title MCC

mcc_list = list(set(df_test.mcc))
df_mcc = pd.DataFrame({'mcc': mcc_list})

zero = df_test[df_test.gender == 0][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
zero = df_mcc.join(pd.DataFrame({'zero': zero}), on='mcc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
one = df_mcc.join(pd.DataFrame({'one': one}), on='mcc', how='outer').fillna(0)

df_mcc = df_mcc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='mcc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='mcc', how='left')

interesting_mcc = set(df_mcc[(df_mcc['count'] > 100) & ((df_mcc['%'] > 70) | (df_mcc['%'] < 30))].mcc)

interesting_mcc

{62, 63, 70, 71, 82, 106, 149, 150, 161}

In [261]:
#@title TTC

ttc_list = list(set(df_test.ttc))
df_ttc = pd.DataFrame({'ttc': ttc_list})

zero = df_test[df_test.gender == 0][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
zero = df_ttc.join(pd.DataFrame({'zero': zero}), on='ttc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
one = df_ttc.join(pd.DataFrame({'one': one}), on='ttc', how='outer').fillna(0)

df_ttc = df_ttc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='ttc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='ttc', how='left')

interesting_ttc = set(df_ttc[(df_ttc['count'] > 100) & ((df_ttc['%'] > 70) | (df_ttc['%'] < 30))].ttc)

interesting_ttc

set()

In [262]:
#@title TID

tid_list = list(set(df_test.tid))
df_tid = pd.DataFrame({'tid': tid_list})

zero = df_test[df_test.gender == 0][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
zero = df_tid.join(pd.DataFrame({'gender': zero}), on='tid', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
one = df_tid.join(pd.DataFrame({'gender': one}), on='tid', how='outer').fillna(0)

df_tid = df_tid.join(zero.gender + one.gender).rename(columns = {'gender': 'count'}).join(zero.gender / (zero.gender + one.gender) * 100).rename(columns = {'gender': '%'})

interesting_tid = set(df_tid[(df_tid['count'] > 100) & ((df_tid['%'] > 70) | (df_tid['%'] < 30))].tid)

len(interesting_tid)
# df_tid[(df_tid['count'] > 100) & ((df_tid['%'] > 70) | (df_tid['%'] < 30))]

5

# **My Code**

In [309]:
#@title Настройки

large_count = 2000
large_sum = -1000000000

k = 1
k1 = 0.8
k2 = 0.85
# k3 = 1.25
k3 = 8

In [310]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count * k1].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum * k1].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 1.07 s, sys: 173 ms, total: 1.24 s
Wall time: 1.73 s


36

In [311]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count * k2].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum * k2) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 999 ms, sys: 196 ms, total: 1.19 s
Wall time: 1.23 s


25

In [312]:
#@title Определение наиболее используемых tid
%%time
tid_by_cid = dfTrx[['cid', 'tid']].drop_duplicates().astype({'tid': str}).groupby('cid').agg({'tid': ' '.join}).sort_values('cid')

large_tid_count = dfTrx[['cid', 'tid']].drop_duplicates().groupby('tid').agg({'cid': 'size'}).sort_values(by = 'cid')
large_tid_sum = dfTrx[['amt', 'tid']].groupby('tid').agg('sum').sort_values(by = 'amt')

most_popular_tid = set(large_tid_count[large_tid_count.cid > large_count * k3].index.to_list()) | set(large_tid_sum[(large_tid_sum.amt < large_sum * k3) | (large_tid_sum.amt > 1000000000)].index.to_list())
len(most_popular_tid)

CPU times: user 5.49 s, sys: 203 ms, total: 5.69 s
Wall time: 5.76 s


0

# **Начало создания df по cid**

In [313]:
#@title Сумма

def s(to_df):
    return to_df / to_df.sum(axis=0)
    # return to_df / np.sqrt((to_df**2).sum(axis=0))
    # return to_df / (np.sqrt(to_df**2)).sum(axis=0)

In [314]:
#@title Настройки

stat = ['count', 'sum', 'median']
is_PCA = 0

In [315]:
%%time
df = df_.copy(deep=True)

CPU times: user 179 µs, sys: 11 µs, total: 190 µs
Wall time: 198 µs


In [316]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

for mcc_index in (most_popular_mcc - interesting_mcc):
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc or interesting_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for mcc_index in interesting_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

to_df = s(to_df) * 200
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 2.1 s, sys: 11.4 ms, total: 2.11 s
Wall time: 2.12 s


In [317]:
#@title TTC
%%time
to_df = df_.copy(deep=True)

for ttc_index in (most_popular_ttc - interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc or interesting_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for ttc_index in (interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.73 s, sys: 5.29 ms, total: 1.74 s
Wall time: 1.75 s


In [318]:
#@title AMT

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [319]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(0, 1):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df) * 2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(1, 9):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')


# to_df = df_.copy(deep=True)

# for hour in range(9, 24):
#     stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
#     to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

# # to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# # to_df.columns += df.shape[1]

# to_df = s(to_df)
# df = df.join(to_df, on = 'cid', how='left')

CPU times: user 3.49 s, sys: 26.5 ms, total: 3.52 s
Wall time: 3.52 s


In [320]:
#@title Date
%%time
to_df = df_.copy(deep=True)

# for weekday in range(7):
#     sum_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
#     to_df = to_df.join(sum_amt_by_cid_and_weekday.amt, on = 'cid', how='left').rename(columns = {'amt': str(weekday) + '_wd'})

for weekday in [0, 3, 4, 5, 6]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')
# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
# to_df = pd.DataFrame(to_df.sum(axis=1))
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')


# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 3), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.68 s, sys: 14.9 ms, total: 1.69 s
Wall time: 1.72 s


In [321]:
#@title TID
# to_df = df_.copy(deep=True)

# for tid_index in (most_popular_tid - interesting_tid):
#     stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
#     to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

# stat_by_other_tid = dfTrx[~dfTrx.tid.isin(most_popular_tid | interesting_tid)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
# stat_by_other_tid.columns += '_tid'
# to_df = to_df.join(stat_by_other_tid, how='left')

# to_df = s(to_df)
# df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for tid_index in (interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [322]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

# **Тест**

In [277]:
a = np.array([1, 2])
b = np.array([2, 4])

(a+b)/2

array([1.5, 3. ])

# **Обучение**

In [327]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)

# estimators = [
#     ('cbc', CBC(iterations=200)),
#     ('lgbmc', LGBMC()),
#     # ('rfc', RFC(n_estimators=10, random_state=42)),
#     # # ('svr', make_pipeline(StandardScaler(),
#     # # ('lr', LR(random_state=0, n_jobs=-1, max_iter=1000)),
#     # ('svc', SVC(probability=True)),
#     # # ('cccv', CCCV(LSVC(C = 3, intercept_scaling = 1, class_weight = 'balanced', random_state = 412), method = 'sigmoid', cv = 5)),
#     ('gbc1', GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)),
#     # ('mlpc', MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1))
# ]

# model = SC(estimators=estimators, cv=2, stack_method='predict_proba', n_jobs=-1,
#            final_estimator=GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2))

model1 = LGBMC()
model2 = CBC(iterations=200)
# model = SGDC()
# model = CCCV(LSVC(C = 3, intercept_scaling = 1, class_weight = 'balanced', random_state = 412), method = 'sigmoid', cv = 5)
# model = LSVC(C = 3, intercept_scaling = 1, class_weight = 'balanced', random_state = 412)

# model = LR(random_state=0, n_jobs=-1, max_iter=1000)
# model3 = RFC()

# model = KNC()
# params = {
#     'n_neighbors': np.arange(1, 11),
#     'metric': ['manhattan', 'euclidean'],
#     'weights': ['uniform', 'distance']
# }
# model = DTC()

# model = SVC(probability=True)
# model = GBC()
model4 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)

# params = {
#     'max_depth': [None, 2, 3, 4, 5],
# }

model5 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)


# model = GSCV(model, params, return_train_score=True, cv=5, scoring='accuracy', n_jobs=-1)

pY1 = model1.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY2 = model2.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
# pY3 = model3.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY4 = model4.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY5 = model5.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
# pY6 = model_6.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]

pY = (pY1 + 2*pY2 + pY4 + pY5) / 5

# pY = model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
# print(model.best_params_)

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))       # fit the model and show test error
print(pY.sum() / pY.shape[0])

Learning rate set to 0.085504
0:	learn: 0.6711555	total: 104ms	remaining: 20.7s
1:	learn: 0.6496162	total: 164ms	remaining: 16.2s
2:	learn: 0.6312705	total: 227ms	remaining: 14.9s
3:	learn: 0.6169535	total: 286ms	remaining: 14s
4:	learn: 0.6018460	total: 346ms	remaining: 13.5s
5:	learn: 0.5902233	total: 402ms	remaining: 13s
6:	learn: 0.5791624	total: 470ms	remaining: 13s
7:	learn: 0.5680270	total: 531ms	remaining: 12.7s
8:	learn: 0.5568187	total: 595ms	remaining: 12.6s
9:	learn: 0.5501028	total: 656ms	remaining: 12.5s
10:	learn: 0.5421403	total: 738ms	remaining: 12.7s
11:	learn: 0.5372617	total: 792ms	remaining: 12.4s
12:	learn: 0.5313725	total: 848ms	remaining: 12.2s
13:	learn: 0.5244987	total: 903ms	remaining: 12s
14:	learn: 0.5199564	total: 963ms	remaining: 11.9s
15:	learn: 0.5157858	total: 1.03s	remaining: 11.8s
16:	learn: 0.5122524	total: 1.09s	remaining: 11.7s
17:	learn: 0.5080864	total: 1.15s	remaining: 11.6s
18:	learn: 0.5035975	total: 1.21s	remaining: 11.5s
19:	learn: 0.499660

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [191]:
tmr.ShowTime()

Runtime is 131 sec > 120 sec limit!!!


In [141]:
# model = LR(random_state=0, n_jobs=-1, max_iter=1000).fit(trainX, trainY.gender)  # now train again, on the full training set
# model = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
# model = SC(estimators=estimators)
# model.fit(trainX, trainY.gender)


model1 = LGBMC()
model2 = CBC(iterations=200)
# model3 = RFC()
model4 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model5 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)

pY1 = model1.fit(trainX, trainY.gender).predict_proba(testX)[:, 1]
pY2 = model2.fit(trainX, trainY.gender).predict_proba(testX)[:, 1]
# pY3 = model3.fit(trainX, trainY.gender).predict_proba(testX0)[:, 1]
pY4 = model4.fit(trainX, trainY.gender).predict_proba(testX)[:, 1]
pY5 = model5.fit(trainX, trainY.gender).predict_proba(testX)[:, 1]
# pY6 = model_6.fit(trainX, trainY.gender).predict_proba(testX0)[:, 1]

pY = (pY1 + pY2 + pY4 + pY5) / 4

Learning rate set to 0.09957
0:	learn: 0.6663657	total: 79.9ms	remaining: 15.9s
1:	learn: 0.6417624	total: 137ms	remaining: 13.6s
2:	learn: 0.6222795	total: 193ms	remaining: 12.7s
3:	learn: 0.6053429	total: 252ms	remaining: 12.3s
4:	learn: 0.5912514	total: 307ms	remaining: 12s
5:	learn: 0.5771944	total: 361ms	remaining: 11.7s
6:	learn: 0.5669891	total: 423ms	remaining: 11.7s
7:	learn: 0.5596556	total: 481ms	remaining: 11.6s
8:	learn: 0.5516980	total: 535ms	remaining: 11.4s
9:	learn: 0.5441714	total: 592ms	remaining: 11.2s
10:	learn: 0.5367728	total: 649ms	remaining: 11.1s
11:	learn: 0.5313270	total: 712ms	remaining: 11.2s
12:	learn: 0.5261092	total: 768ms	remaining: 11s
13:	learn: 0.5212550	total: 823ms	remaining: 10.9s
14:	learn: 0.5166626	total: 877ms	remaining: 10.8s
15:	learn: 0.5111905	total: 936ms	remaining: 10.8s
16:	learn: 0.5062143	total: 991ms	remaining: 10.7s
17:	learn: 0.5026486	total: 1.05s	remaining: 10.6s
18:	learn: 0.4975706	total: 1.11s	remaining: 10.6s
19:	learn: 0.49

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [142]:
pY = pd.DataFrame(pY, columns=['gender'])
pY.index.name = 'cid'

print(pY.sum() / pY.shape[0])

gender    0.444264
dtype: float64


In [143]:
pY

,gender
cid,
0,0.587919
1,0.244309
...,...
1998,0.645957
1999,0.917383


In [133]:
pd.read_csv('💳Baseline (11).csv')

,cid,gender
0,0,0.410756
1,1,0.367415
...,...,...
1998,1998,0.730247
1999,1999,0.956158


In [144]:
pY.reset_index().to_csv('💳Baseline.csv', index_label='cid', index=False, float_format='%10f')